# Solution steps below:

---

1. Count frequency and probability of every letter on ASCII from book: /media/book.txt


In [29]:
from collections import Counter

with open("media/book.txt", "r", encoding="utf-8") as f:
    text = f.read().lower()

counter = Counter(
    ch for ch in text
    if 'a' <= ch <= 'z'
)

alphabet = dict(sorted(counter.items()))

total_letters = sum(alphabet.values())

print("---- Count of letters ----")

for letter, cnt in alphabet.items():
    print(letter, cnt)


---- Count of letters ----
a 22277
b 4175
c 6467
d 12728
e 36216
f 6575
g 5814
h 19531
i 18348
j 403
k 2225
l 11428
m 6419
n 18741
o 20431
p 4833
q 239
r 16196
s 17599
t 26744
u 7656
v 2718
w 6950
x 313
y 5269
z 136


In [30]:
print("--- Probabality of each letter ---")
probabilities = {
    letter: count / total_letters
    for letter, count in alphabet.items()
}

for letter, prob in probabilities.items():
    print(letter, round(prob, 4))

--- Probabality of each letter ---
a 0.0794
b 0.0149
c 0.0231
d 0.0454
e 0.1291
f 0.0234
g 0.0207
h 0.0696
i 0.0654
j 0.0014
k 0.0079
l 0.0408
m 0.0229
n 0.0668
o 0.0729
p 0.0172
q 0.0009
r 0.0578
s 0.0628
t 0.0954
u 0.0273
v 0.0097
w 0.0248
x 0.0011
y 0.0188
z 0.0005


2. Create Huffman Code encoding the chosen book. And save on media/output.txt 

In [31]:

import heapq

# creating binary tree structure
# with min heap to get minimal probabilities
# and merge them after that again run heap 
# and generate code for each letter with taken probailities


class Node:
    def __init__(self, char=None, freq=0):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(frequency_map):
    heap = []
    for char, freq in frequency_map.items():
        heapq.heappush(heap, Node(char, freq))

    if not heap:
        return None

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        merged = Node(freq=left.freq + right.freq)
        merged.left = left
        merged.right = right

        heapq.heappush(heap, merged)

    return heap[0]


def generate_codes(node, current_code="", codes=None):
    if codes is None:
        codes = {}

    if node is None:
        return codes

    if node.char is not None:
        codes[node.char] = current_code

    generate_codes(node.left, current_code + "0", codes)
    generate_codes(node.right, current_code + "1", codes)

    return codes


tree = build_huffman_tree(alphabet) 
codes = generate_codes(tree)

print("--- Generated Codes in sorted format ---")

codes = dict(sorted(codes.items()))



for char, code in codes.items():
    print(char, code)


with open("outputs/codes.txt", "w", encoding="utf-8") as f:
    for char, code in codes.items():
        line = char + " " + code + "\n"
        f.writelines(line)


print("--- Codes on sorted format saved on outputs/codes.txt ---")    






--- Generated Codes in sorted format ---
a 1110
b 110000
c 00010
d 0000
e 100
f 00011
g 110011
h 1011
i 0111
j 111110100
k 11111011
l 11110
m 111111
n 1010
o 1101
p 110001
q 11111010111
r 0101
s 0110
t 001
u 01001
v 1111100
w 01000
x 1111101010
y 110010
z 11111010110
--- Codes on sorted format saved on outputs/codes.txt ---


3. Checking satisfiying on Kraff-McMillian inequality



In [32]:
def check_kraft_inequality(codes):
    print("--- Kraft-McMillan Inequality Check ---")
    
    k_sum = 0
    print(f"{'Char':<10} | {'Code':<15} | {'Length':<5} | {'2^(-L)':<10}")
    print("-" * 40)
    
    for char, code in codes.items():
        length = len(code)
        term = 2 ** (-length)
        k_sum += term
        print(f"{char:<10} | {code:<15} | {length:<5} | {term:.9f}")

    print("-" * 40)
    print(f"Total Sum (K): {k_sum}")

    if k_sum <= 1.0 + 1e-9:
        print("Result: Inequality SATISFIED (K <= 1)")
        if abs(k_sum - 1.0) < 1e-9:
            print("Note: The code is complete (K = 1).")
    else:
        print("Result: Inequality NOT Satisfied (Something is wrong with the tree).")



check_kraft_inequality(codes)

--- Kraft-McMillan Inequality Check ---
Char       | Code            | Length | 2^(-L)    
----------------------------------------
a          | 1110            | 4     | 0.062500000
b          | 110000          | 6     | 0.015625000
c          | 00010           | 5     | 0.031250000
d          | 0000            | 4     | 0.062500000
e          | 100             | 3     | 0.125000000
f          | 00011           | 5     | 0.031250000
g          | 110011          | 6     | 0.015625000
h          | 1011            | 4     | 0.062500000
i          | 0111            | 4     | 0.062500000
j          | 111110100       | 9     | 0.001953125
k          | 11111011        | 8     | 0.003906250
l          | 11110           | 5     | 0.031250000
m          | 111111          | 6     | 0.015625000
n          | 1010            | 4     | 0.062500000
o          | 1101            | 4     | 0.062500000
p          | 110001          | 6     | 0.015625000
q          | 11111010111     | 11    | 0.000488281
r

In [33]:

def encode_text(text, codes):
    encoded = ""
    for ch in text.lower():
        if ch in codes:
            encoded += codes[ch]
    return encoded

print("--- Proccess Saving a encoding text to outputs/output.txt ")

encoded_text = encode_text(text, codes)

with open("outputs/encoded_book.txt", "w", encoding="utf-8") as f:
    f.write(encoded_text)

print(f"Huffman encoding complete. Output length: {len(encoded_text)} bits.")

--- Proccess Saving a encoding text to outputs/output.txt 
Huffman encoding complete. Output length: 1174432 bits.
